In [19]:
import numpy as np
import pandas as pd
import pickle
import xgboost as xgb

In [11]:
with open('../data/output_data/model.pickle', 'rb') as f:
    model = pickle.load(f)
with open('../data/output_data/train_df.pickle', 'rb') as f:
    train_df: pd.DataFrame = pickle.load(f)
train_X = train_df.drop('author_id', axis=1)
dtrain = xgb.DMatrix(train_X)

In [27]:
pred = model.predict(dtrain)
np.argmax(pred, axis=1)

array([839, 839, 224, ..., 842, 842, 554], dtype=int64)